In [1]:
import pandas as pd
import numpy as np
import json
import os
from tabulate import tabulate

In [2]:
import git
repo_path = git.Repo('.', search_parent_directories=True).working_tree_dir

In [10]:
folder_path = os.path.join(repo_path, r"SUSY\SUSY QM\PennyLane\COBYQA\PauliDecomp\VQD\Files\RA-Linear2\10000")
cutoffs = [2,4,8,16,32]
potentials = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

In [11]:
converged_only = True
all_data = []

for potential in potentials:
    for cutoff in cutoffs:

        median = []
        mean = []
        min = []
        e0 = []
        e1 = []
        e2 = []
        ratios = []

        data_path = folder_path + "\\{}\\{}_{}.json".format(potential,potential, cutoff)
        
        with open(data_path, 'r') as file:
            data = json.load(file)

        num_VQD = data['num_VQD']

        converged = [all(x) for x in data['success']]
        converged_indices = [i for i, success in enumerate(converged) if success]
        converged_runs = sum(converged)

        if converged_only:
            results = [data['results'][i] for i in converged_indices]
        else:
            results = data['results']

        success_i = [i for i, success in enumerate(data['success']) if all(success)]
        successful_results = [data['results'][i] for i in success_i]
        energies =np.sort(successful_results)

        
        run_times = data['run_times']
        time_series = np.array([pd.Timedelta(t).total_seconds() for t in run_times])
        mean_vqe_time = np.mean(time_series)
        time_taken = np.mean(mean_vqe_time)

        num_evals = data['num_evaluations']
        sum_evals = [sum(x) for x in num_evals]
        mean_evals = np.mean(sum_evals)

        num_iters = data['num_iters']
        sum_iters = [sum(x) for x in num_iters]
        mean_iters = np.mean(sum_iters)

        for e in np.sort(energies):
            e0.append(e[0])
            e1.append(e[1])
            e2.append(e[2])

        mean_ratio = abs((np.mean(e2) - np.mean(e1)) / (np.mean(e2) - np.mean(e0)))
        median_ratio = abs((np.median(e2) - np.median(e1)) / (np.median(e2) - np.median(e0)))
        #ratios.append(r)

        e0_exact = data['exact_eigenvalues'][0]
        e1_exact = data['exact_eigenvalues'][1]
        e2_exact = data['exact_eigenvalues'][2]
        exact_ratio = abs((e2_exact - e1_exact) / (e2_exact - e0_exact))

        row = {
        "potential": potential,
        "cutoff": cutoff,
        'Converged Runs': f"{converged_runs}/100",
        r'$N_{\text{iters}}$': int(mean_iters),
        r'$N_{\text{evals}}$': int(mean_evals),
        #'Mean VQD time (s)': f"{time_taken:.2f}",
        "e0": np.median(e0),
        "e1": np.median(e1),
        "e2": np.median(e2),
        "median_ratio": median_ratio,
        #"mean_ratio": mean_ratio,
        #"min_ratio": np.min(ratios),
        "e0_exact": e0_exact,
        "e1_exact": e1_exact,
        "e2_exact": e2_exact,
        "exact_ratio": exact_ratio
        }

        all_data.append(row)

In [12]:
df = pd.DataFrame(all_data)
df

,potential,cutoff,Converged Runs,$N_{\text{iters}}$,$N_{\text{evals}}$,e0,e1,e2,median_ratio,e0_exact,e1_exact,e2_exact,exact_ratio
0,AHO,2,100/100,166,553,-5.390000e-01,-0.437500,1.902500,0.958427,-0.437500,-0.437500,2.062500,1.000000e+00
1,AHO,4,100/100,237,1328,-2.273259e-01,1.351007,2.421483,0.404135,-0.164785,0.673310,1.667943,5.427061e-01
2,AHO,8,100/100,268,3244,3.040549e-03,1.535902,5.047915,0.696155,0.032010,1.680156,1.833526,8.513391e-02
3,AHO,16,100/100,310,9105,1.234689e-01,1.943847,5.824457,0.680691,-0.001167,1.677494,1.686381,5.266251e-03
4,AHO,32,100/100,362,23094,8.467879e-01,3.917751,10.046683,0.666196,0.000006,1.686499,1.686559,3.585457e-05
5,DW,2,100/100,190,627,3.572330e-01,0.771447,1.336802,0.577147,0.357233,0.771447,1.478553,6.306019e-01
6,DW,4,100/100,235,2656,8.939637e-01,1.053739,2.011760,0.857063,0.906560,0.950634,1.695666,9.441474e-01
7,DW,8,100/100,280,8347,6.627542e-01,1.937895,3.941417,0.611079,0.884580,0.887725,2.693873,9.982617e-01
8,DW,16,100/100,322,21898,2.953336e-01,2.563435,5.661357,0.577322,0.891599,0.891641,2.734122,9.999774e-01
9,DW,32,100/100,366,53205,6.817930e-02,4.268817,7.266484,0.416441,0.891632,0.891632,2.734011,1.000000e+00


In [13]:
df = pd.DataFrame(all_data)
#df[r'$\DeltaE$'] = df[['median_ratio','exact_ratio']].apply(lambda x: np.abs(x['median_ratio'] - x['exact_ratio']), axis=1)
df.to_latex(os.path.join(folder_path, f"ratios.tex"), index=False, float_format="%.4f")

In [ ]:
df = pd.DataFrame(all_data)
df['diff'] = df[['median_ratio','exact_ratio']].apply(lambda x: np.abs(x['median_ratio'] - x['exact_ratio']), axis=1)
df

In [20]:
df = pd.DataFrame(all_data)
df['diff'] = df[['median_ratio','exact_ratio']].apply(lambda x: np.abs(x['median_ratio'] - x['exact_ratio']), axis=1)
df

,potential,cutoff,Converged Runs,$N_{\text{evals}}$,e0,e1,e2,median_ratio,e0_exact,e1_exact,e2_exact,exact_ratio,diff
0,AHO,2,100/100,826,-0.437500,-0.129625,3.615750,0.924042,-0.437500,-0.437500,2.062500,1.000000e+00,0.075958
1,AHO,4,100/100,1917,0.564413,1.579444,5.133909,0.777868,-0.164785,0.673310,1.667943,5.427061e-01,0.235162
2,AHO,8,100/100,4440,9.324501,15.624757,23.355967,0.550991,0.032010,1.680156,1.833526,8.513391e-02,0.465857
3,AHO,16,100/100,12557,60.228340,124.492069,215.306400,0.585604,-0.001167,1.677494,1.686381,5.266251e-03,0.580338
4,AHO,32,100/100,31282,669.913500,1158.075620,1994.004960,0.631323,0.000006,1.686499,1.686559,3.585457e-05,0.631287
5,DW,2,100/100,835,0.361048,1.144404,4.683263,0.818760,0.357233,0.771447,1.478553,6.306019e-01,0.188158
6,DW,4,100/100,3657,1.343495,1.993792,3.789334,0.734121,0.906560,0.950634,1.695666,9.441474e-01,0.210026
7,DW,8,100/100,11400,3.428074,6.786952,11.413304,0.579364,0.884580,0.887725,2.693873,9.982617e-01,0.418898
8,DW,16,100/100,30167,10.937471,16.099414,24.433867,0.617532,0.891599,0.891641,2.734122,9.999774e-01,0.382446
9,DW,32,100/100,74479,33.638877,48.953910,71.000388,0.590085,0.891632,0.891632,2.734011,1.000000e+00,0.409915


In [8]:
df = pd.DataFrame(all_data)
df['diff'] = df[['median_ratio','exact_ratio']].apply(lambda x: np.abs(x['median_ratio'] - x['exact_ratio']), axis=1)
df

,potential,cutoff,Converged Runs,Mean VQD time (s),e0,e1,e2,e0_exact,e1_exact,e2_exact,median_ratio,exact_ratio,diff
0,AHO,2,100/100,0.64,-4.375000e-01,-4.375000e-01,-0.436246,-0.437500,-0.437500,2.062500,9.999633e-01,1.000000e+00,3.671680e-05
1,AHO,4,100/100,2.39,-1.647842e-01,6.733109e-01,1.656297,-0.164785,0.673310,1.667943,5.397817e-01,5.427061e-01,2.924375e-03
2,AHO,8,100/100,21.85,4.160560e-02,1.681011e+00,2.083244,0.032010,1.680156,1.833526,1.970148e-01,8.513391e-02,1.118809e-01
3,AHO,16,100/100,200.78,3.962557e-02,1.716024e+00,2.861201,-0.001167,1.677494,1.686381,4.058646e-01,5.266251e-03,4.005983e-01
4,DW,2,100/100,0.98,3.572331e-01,5.204717e-01,0.791603,0.357233,0.771447,1.478553,6.241943e-01,6.306019e-01,6.407598e-03
5,DW,4,100/100,5.97,9.080980e-01,1.027603e+00,1.696102,0.906560,0.950634,1.695666,8.483451e-01,9.441474e-01,9.580224e-02
6,DW,8,100/100,30.87,8.951463e-01,1.090723e+00,2.983337,0.884580,0.887725,2.693873,9.063416e-01,9.982617e-01,9.192015e-02
7,DW,16,100/100,188.93,8.961524e-01,1.091884e+00,3.144082,0.891599,0.891641,2.734122,9.129282e-01,9.999774e-01,8.704927e-02
8,QHO,2,100/100,0.85,5.135977e-10,5.296452e-08,0.046213,0.000000,0.000000,1.000000,9.999989e-01,1.000000e+00,1.134991e-06
9,QHO,4,100/100,1.45,1.255998e-07,1.000000e+00,1.000000,0.000000,1.000000,1.000000,2.001772e-07,0.000000e+00,2.001772e-07


In [19]:
latex_table = tabulate(df.values, headers=df.columns, tablefmt="latex", floatfmt=".3f")
print(latex_table)

\begin{tabular}{lrlrrrrrrrrrr}
\hline
 potential   &   cutoff & Converged Runs   &   Mean VQD time (s) &     e0 &     e1 &    e2 &   median\_ratio &   mean\_ratio &   e0\_exact &   e1\_exact &   e2\_exact &   exact\_ratio \\
\hline
 AHO         &        2 & 100/100          &              96.260 & -0.485 & -0.437 & 1.472 &          0.976 &        0.977 &     -0.437 &     -0.437 &      2.062 &         1.000 \\
 AHO         &        4 & 100/100          &             110.610 & -0.165 &  0.632 & 1.613 &          0.552 &        0.550 &     -0.165 &      0.673 &      1.668 &         0.543 \\
 AHO         &        8 & 100/100          &             152.230 &  0.032 &  1.647 & 1.796 &          0.086 &        0.089 &      0.032 &      1.680 &      1.834 &         0.085 \\
 AHO         &       16 & 97/100           &             296.420 & -0.001 &  1.652 & 1.670 &          0.017 &        0.059 &     -0.001 &      1.677 &      1.686 &         0.005 \\
 AHO         &       32 & 94/100           &